<a href="https://colab.research.google.com/github/rahiakela/deep-learning-for-time-series-forecasting/blob/part-3-deep-learning-methods/2_time_series_forecasting_using_multi_layer_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series Forecasting using Multi-Layer Perceptron

Multilayer Perceptrons, or MLPs for short, can be applied to time series forecasting. A challenge with using MLPs for time series forecasting is in the preparation of the data. Specifically, lag observations must be  attened into feature vectors. 

In this notebook, we will discover how to develop a suite of Multilayer Perceptron models for a range of standard time series forecasting problems.

* Develop MLP models for univariate time series forecasting.
* Develop MLP models for multivariate time series forecasting.
* Develop MLP models for multi-step time series forecasting.

So the notebook is divided into four parts; they are:

1. Univariate MLP Models
2. Multivariate MLP Models
3. Multi-step MLP Models
4. Multivariate Multi-step MLP Models

Traditionally, a lot of research has been invested into using MLPs for time series forecasting with modest results. Perhaps the most promising area in the application of deep learning methods to time series forecasting are in the use of CNNs, LSTMs and hybrid models. As such, we will not see more examples of straight MLP models for time series forecasting.

## Setup

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

## Univariate MLP Models

Multilayer Perceptrons, or MLPs for short, can be used to model univariate time series forecasting problems. Univariate time series are a dataset comprised of a single series of observations with a temporal ordering and a model is required to learn from the series of past observations to predict the next value in the sequence. This section is divided into two parts; they are:

1. Data Preparation
2. MLP Model


### Data Preparation

Before a univariate series can be modeled, it must be prepared. The MLP model will learn a function that maps a sequence of past observations as input to an output observation. As such, the sequence of observations must be transformed into multiple examples from which the model can learn.

```python
[10, 20, 30, 40, 50, 60, 70, 80, 90]
```

We can divide the sequence into multiple input/output patterns called samples, where three time steps are used as input and one time step is used as output for the one-step prediction that is being learned.

```python
X,          y
10, 20, 30, 40
20, 30, 40, 50
30, 40, 50, 60
..............
```

The split sequence() function below implements this behavior and will split a given univariate sequence into multiple samples where each sample has a specified number of time steps and the output is a single time step.


In [0]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence) - 1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return np.array(X), np.array(y)

In [3]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# choose a number of time steps
n_steps = 3

# split into samples
X, y = split_sequence(raw_seq, n_steps)

# summarize the data
for i in range(len(X)):
  print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


Now that we know how to prepare a univariate series for modeling, let's look at developing an MLP model that can learn the mapping of inputs to outputs.

### MLP Model

A simple MLP model has a single hidden layer of nodes, and an output layer used to make a prediction.

We almost always have multiple samples, therefore, the model will expect the input component of training data to have the dimensions or shape: $[samples, features]$.

The model expects the input shape to be two-dimensional with $[samples,
features]$, therefore, we must reshape the single input sample before making the prediction, e.g with the shape [1, 3] for 1 sample and 3 time steps used as input features.

We can make this concept concrete with a worked example.


In [5]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps))
model.add(Dense(1))

# compile model'
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=200, verbose=0)

# demonstrate prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[108.95295]]


## Multivariate MLP Models

Consider that you are in the current situation:

``
I have two columns in my data file with 5,000 rows, column 1 is time (with 1 hour interval) and column 2 is the number of sales and I am trying to forecast the number of sales for future time steps. Help me to set the number of samples, time steps and features in this data for an LSTM?
``

There are few problems here:

* **Data Shape**: LSTMs expect 3D input, and it can be challenging to get your head around this the first time.
* **Sequence Length**: LSTMs don't like sequences of more than 200-400 time steps, so the data will need to be split into subsamples.

We will work through this example, broken down into the following 4 steps:

1. Load the Data
2. Drop the Time Column
3. Split Into Samples
4. Reshape Subsequences



### Load the Data

In [0]:
# load time series dataset
# series = pd.read_csv('filename.csv', header=0, index_col=0)

# We will mock loading by defining a new dataset in memory with 5,000 time steps.
# define the dataset
data = list()
n = 5000
for i in range(n):
  data.append([i+1, (i+1) * 10])
data = np.array(data)
print(data[:5, :])
print(data.shape)

[[ 1 10]
 [ 2 20]
 [ 3 30]
 [ 4 40]
 [ 5 50]]
(5000, 2)


We can see we have 5,000 rows and 2 columns: a standard univariate time series dataset.

### Drop the Time Column

If your time series data is uniform over time and there is no missing values, we can drop the time column. If not, you may want to look at imputing the missing values, resampling the data to a new time scale, or developing a model that can handle missing values. 

Here, we just drop the first column:

In [0]:
# define the dataset
data = list()
n = 5000
for i in range(n):
  data.append([i+1, (i+1) * 10])
data = np.array(data)

# drop time
data = data[:, 1]
print(data.shape)

(5000,)


### Split Into Samples

LSTMs need to process samples where each sample is a single sequence of observations. In this case, 5,000 time steps is too long; LSTMs work better with 200-to-400 time steps. Therefore, we need to split the 5,000 time steps into multiple shorter sub-sequences.

For example, perhaps you need overlapping sequences, perhaps non-overlapping is good but your model needs state across the sub-sequences and so on. 

In this example, we will split the 5,000 time steps into 25 sub-sequences of 200 time steps each.

In [0]:
# define the dataset
data = list()
n = 5000
for i in range(n):
  data.append([i+1, (i+1) * 10])
data = np.array(data)

# drop time
data = data[:, 1]
print(data.shape)

# split into samples (e.g. 5000/200 = 25)
samples = list()
length = 200

# step over the 5,000 in jumps of 200
for i in range(0, n , length):
  sample = data[i: i + length]  # grab from i to i + 200
  samples.append(sample)
print(len(samples))

(5000,)
25


In [0]:
len(samples[:5][0])

200

In [0]:
samples[:2]

[array([  10,   20,   30,   40,   50,   60,   70,   80,   90,  100,  110,
         120,  130,  140,  150,  160,  170,  180,  190,  200,  210,  220,
         230,  240,  250,  260,  270,  280,  290,  300,  310,  320,  330,
         340,  350,  360,  370,  380,  390,  400,  410,  420,  430,  440,
         450,  460,  470,  480,  490,  500,  510,  520,  530,  540,  550,
         560,  570,  580,  590,  600,  610,  620,  630,  640,  650,  660,
         670,  680,  690,  700,  710,  720,  730,  740,  750,  760,  770,
         780,  790,  800,  810,  820,  830,  840,  850,  860,  870,  880,
         890,  900,  910,  920,  930,  940,  950,  960,  970,  980,  990,
        1000, 1010, 1020, 1030, 1040, 1050, 1060, 1070, 1080, 1090, 1100,
        1110, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200, 1210,
        1220, 1230, 1240, 1250, 1260, 1270, 1280, 1290, 1300, 1310, 1320,
        1330, 1340, 1350, 1360, 1370, 1380, 1390, 1400, 1410, 1420, 1430,
        1440, 1450, 1460, 1470, 1480, 

We now have 25 subsequences of 200 time steps each.

### Reshape Subsequences

The LSTM needs data with the format of $[samples, timesteps, features]$. We have 25 samples, 200 time steps per sample, and 1 feature. 

First, we need to convert our list of arrays into a 2D NumPy array with the shape $[25, 200]$.

In [0]:
# define the dataset
data = list()
n = 5000
for i in range(n):
  data.append([i+1, (i+1) * 10])
data = np.array(data)

# drop time
data = data[:, 1]
print(data.shape)

# split into samples (e.g. 5000/200 = 25)
samples = list()
length = 200

# step over the 5,000 in jumps of 200
for i in range(0, n , length):
  sample = data[i: i + length]  # grab from i to i + 200
  samples.append(sample)
print(len(samples))

# convert list of arrays into 2d array
data = np.array(samples)
print(data.shape)

(5000,)
25
(25, 200)


Now we have 25 rows and 200 columns. Interpreted in a machine learning context, this dataset has 25 samples and 200 features per sample.

Next, we can use the reshape() function to add one additional dimension for our single feature and use the existing columns as time steps instead.

In [0]:
# reshape into [samples, timesteps, features]
data = data.reshape((len(samples), length, 1))
print(data.shape)

(25, 200, 1)


And that is it. The data can now be used as an input (X) to an LSTM model, or even a CNN model.